In [1]:
from recoxplainer.config import cfg
from recoxplainer.data_reader.data_reader import DataReader
from recoxplainer.models import GMFModel, ALS, EMFModel, BPR
from recoxplainer.recommender import Recommender
from recoxplainer.evaluator import Splitter, Evaluator
from recoxplainer.explain import ARPostHocExplainer 
from recoxplainer.evaluator import ExplanationEvaluator
from recoxplainer.models.mlp_model import MLPModel
import pandas as pd
pd.options.mode.chained_assignment = None

In [2]:
data = DataReader(**cfg.ml100k)
data.make_consecutive_ids_in_dataset()
data.binarize()
sp = Splitter()
train, test = sp.split_leave_n_out(data, frac=0.1)

In [3]:
als = ALS(**cfg.model.als)
als.fit(data)

/Users/mac/opt/anaconda3/envs/recoxplainer/lib/python3.6/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


  0%|          | 0/10 [00:00<?, ?it/s]

True

In [4]:
rec = Recommender(data, als)
recommendations = rec.recommend_all()

Recommending for users:   0%|          | 0/943 [00:00<?, ?it/s]

In [5]:
recommendations.head()

,userId,itemId,rank
161,0.0,166.0,1.0
7,0.0,8.0,2.0
85,0.0,89.0,3.0
387,0.0,404.0,4.0
128,0.0,133.0,5.0


## AR explainer

In [6]:
ARexplainer = ARPostHocExplainer(als, recommendations, data)
ARexpl = ARexplainer.explain_recommendations()

Computing explanations:   0%|          | 0/9430 [00:00<?, ?it/s]

In [7]:
ARexpl.head()

,userId,itemId,rank,explanations
161,0.0,166.0,1.0,"{289, 321, 291, 389, 423, 329, 10, 363, 431, 4..."
7,0.0,8.0,2.0,"{321, 423, 329, 431, 528, 179, 86, 221}"
85,0.0,89.0,3.0,"{512, 289, 321, 389, 423, 329, 10, 491, 432, 8..."
387,0.0,404.0,4.0,"{289, 491}"
128,0.0,133.0,5.0,"{512, 289}"


## Evaluation 

In [8]:
ex = ExplanationEvaluator(data.num_user)

In [9]:
ex.model_fidelity(ARexpl)

0.8094379639448539